# Experiment Runner Wrapper - WORKING VERSION

This notebook demonstrates the comprehensive experiment runner wrapper that integrates all modules and orchestrates complete experimental pipelines for autoencoder research.

## Test Steps:
1. **Import libraries and test basic functionality**
2. **Initialize the wrapper**
3. **Test dataset preparation**
4. **Run a simple experiment**
5. **Analyze results**

Let's start testing step by step...

In [ ]:
# Step 1: Import libraries and test basic functionality
print("=== STEP 1: TESTING IMPORTS ===")

import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
import json
import time
from datetime import datetime
from typing import Dict, List, Any, Optional, Tuple, Union
from torch.utils.data import DataLoader, TensorDataset

# Import autoencoder library modules
from autoencoder_lib.experiment import ExperimentRunner
from autoencoder_lib.models import create_autoencoder, MODEL_ARCHITECTURES
from autoencoder_lib.data import generate_dataset

print("✅ All imports successful!")
print(f"Available model architectures: {list(MODEL_ARCHITECTURES.keys())}")
print(f"PyTorch version: {torch.__version__}")
print(f"Device available: {torch.device('cuda' if torch.cuda.is_available() else 'cpu')}")

In [ ]:
# Step 2: Define the ExperimentRunnerWrapper class
print("=== STEP 2: DEFINING WRAPPER CLASS ===")

class ExperimentRunnerWrapper:
    \"\"\"
    High-level wrapper for systematic autoencoder experimentation.
    \"\"\"
    
    def __init__(self, 
                 output_dir: str = "experiment_results",
                 device: Optional[torch.device] = None,
                 random_seed: int = 42,
                 verbose: bool = True):
        \"\"\"Initialize the ExperimentRunnerWrapper.\"\"\"
        self.output_dir = Path(output_dir)
        self.output_dir.mkdir(parents=True, exist_ok=True)
        
        self.device = device if device is not None else torch.device(
            "cuda" if torch.cuda.is_available() else "cpu"
        )
        self.random_seed = random_seed
        self.verbose = verbose
        
        # Initialize the base experiment runner
        self.experiment_runner = ExperimentRunner(
            device=self.device,
            output_dir=str(self.output_dir),
            random_seed=random_seed
        )
        
        if self.verbose:
            print(f"ExperimentRunnerWrapper initialized")
            print(f"  Device: {self.device}")
            print(f"  Output directory: {self.output_dir}")
    
    def prepare_simple_dataset(self, dataset_config: Dict[str, Any]) -> Tuple[DataLoader, torch.Tensor, torch.Tensor, List[str]]:
        \"\"\"Generate dataset and prepare it for training.\"\"\"
        if self.verbose:
            print("Preparing dataset...")
        
        # Generate dataset
        dataset_info = generate_dataset(**dataset_config)
        
        # Simple data loading (for demonstration)
        from PIL import Image
        import os
        
        output_dir = dataset_config['output_dir']
        class_names = dataset_info['label_names']
        
        # Load all data
        all_data = []
        all_labels = []
        
        # Process each class
        for class_idx, class_name in enumerate(class_names):
            class_dir = Path(output_dir) / class_name
            
            if class_dir.exists():
                for img_file in class_dir.glob("*.png"):
                    img = Image.open(img_file).convert('L')  # Grayscale
                    img_array = np.array(img, dtype=np.float32) / 255.0
                    all_data.append(img_array)
                    all_labels.append(class_idx)
        
        # Convert to tensors
        all_data = torch.tensor(np.array(all_data), dtype=torch.float32).unsqueeze(1)  # Add channel dim
        all_labels = torch.tensor(all_labels, dtype=torch.long)
        
        # Simple train/test split
        split_point = int(len(all_data) * 0.8)
        train_data = all_data[:split_point]
        train_labels = all_labels[:split_point]
        test_data = all_data[split_point:]
        test_labels = all_labels[split_point:]
        
        # Create DataLoader for training
        train_dataset = TensorDataset(train_data, train_data, train_labels)  # (x, y, labels) format
        train_loader = DataLoader(
            train_dataset, 
            batch_size=dataset_config.get('batch_size', 8),
            shuffle=True
        )
        
        if self.verbose:
            print(f"Dataset prepared:")
            print(f"  Train samples: {len(train_data)}")
            print(f"  Test samples: {len(test_data)}")
            print(f"  Classes: {class_names}")
            print(f"  Image shape: {train_data.shape[1:]}")
        
        return train_loader, test_data, test_labels, class_names

print("✅ ExperimentRunnerWrapper class defined!")

In [ ]:
# Step 3: Initialize the wrapper and test dataset preparation
print("=== STEP 3: TESTING WRAPPER INITIALIZATION ===")

# Initialize the wrapper
wrapper = ExperimentRunnerWrapper(
    output_dir="test_wrapper_results",
    random_seed=42,
    verbose=True
)

print("\\n=== TESTING DATASET PREPARATION ===")

# Define a simple dataset configuration
dataset_config = {
    'dataset_type': 'layered_geological',
    'output_dir': 'test_wrapper_dataset',
    'num_samples_per_class': 8,  # Small for testing
    'image_size': 32,
    'num_classes': 2,
    'batch_size': 4
}

# Test dataset preparation
try:
    train_loader, test_data, test_labels, class_names = wrapper.prepare_simple_dataset(dataset_config)
    print(f"\\n✅ Dataset preparation successful!")
    print(f"   Train loader batches: {len(train_loader)}")
    print(f"   Test samples: {len(test_data)}")
    print(f"   Classes: {class_names}")
    print(f"   Test data shape: {test_data.shape}")
except Exception as e:
    print(f"❌ Dataset preparation failed: {e}")
    import traceback
    traceback.print_exc()

In [ ]:
# Step 4: Test model creation and simple training
print("=== STEP 4: TESTING MODEL CREATION AND TRAINING ===")

# Test model creation
print("Testing model creation...")
model = create_autoencoder(
    architecture_name='simple_linear',
    input_size=32*32,  # For 32x32 images
    latent_dim=16
)
print(f"✅ Model created: {type(model).__name__}")

# Test a very short training run
print("\\nTesting short training run...")
try:
    trained_model, history = wrapper.experiment_runner.train_autoencoder(
        model=model,
        train_loader=train_loader,
        test_data=test_data,
        test_labels=test_labels,
        epochs=2,  # Very short for testing
        learning_rate=0.001,
        class_names=class_names,
        save_model=False,  # Don't save for testing
        experiment_name=\"test_experiment\"
    )
    
    print(f"\\n✅ Training completed successfully!")
    print(f"   Final train loss: {history.get('final_train_loss', 'N/A')}")
    print(f"   Final test loss: {history.get('final_test_loss', 'N/A')}")
    print(f"   Training time: {history.get('training_time', 'N/A'):.2f}s")
    
except Exception as e:
    print(f"❌ Training failed: {e}")
    import traceback
    traceback.print_exc()

# Test Results Summary

Run the cells above in order to test the wrapper functionality:

1. **Cell 1**: Test imports - should show all modules loading correctly
2. **Cell 2**: Define wrapper class - creates the main ExperimentRunnerWrapper  
3. **Cell 3**: Initialize wrapper and test dataset - creates small test dataset
4. **Cell 4**: Test model creation and training - runs a 2-epoch training test

## Expected Behavior:
- All imports should work without errors
- Dataset generation should create 8 samples per class (16 total)
- Model creation should succeed for 'simple_linear' architecture  
- Training should complete in a few seconds with loss values

## Common Issues to Watch For:
- Import errors (we fixed these in the runner.py file)
- Architecture name mismatches (use names from MODEL_ARCHITECTURES)
- Memory issues (we use small datasets for testing)
- Visualization backend issues (matplotlib setup)

If all cells run successfully, the wrapper is working correctly! 🎉